## Transform eta maps to javascript usable format
This notebook reads in h/eta values from h_eta_small.npy files and aggregates them as javascript variables in a single file.

### First import some necessary tools


In [1]:
%pylab inline
from __future__ import print_function
from ptha_paths import data_dir, events_dir, js_data_dir
import sys, os

Populating the interactive namespace from numpy and matplotlib
Assuming that top level of this repository is at: /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial
    Python codes can be found in codes_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/PythonCode
    Data files can be found in data_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/DataFiles
    Results for each event can be found in events_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/ptha_tutorial/DataFiles/Events
Assuming that top level of the CSE512 repository is at: /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation
    JavaScript data files can be found in data_dir = /media/brian/TBdrive/Documents/Coursework/CSE512/Homework/project/tsunami-inundation/docs/data


### Set parameters 

In [4]:
# The events to consider
events = ['AASZa', 'AASZb', 'AASZc', 'AASZd', 'CSZa', 'CSZb', 'CSZc', 'CSZd', 'CSZe', \
              'CSZf', 'KmSZa', 'KrSZa', 'SChSZa', 'TOHa']

# Resolution of the data
nx = 250
ny = 250

# Name of file where we will output results
out_filename = 'eta_arrs.js'


### Loop over all events and output zeta to a javascript file

In [12]:
# Get location of file where we'll output the results
out_file = os.path.join(js_data_dir,out_filename)

first_event = True
for event in events:
    
    # Load the eta data file
    event_dir = os.path.join(events_dir, event)
    hmax_file = os.path.join(event_dir, 'h_eta_small.npy')
    hmax      = load(hmax_file)
    Hmax      = hmax.reshape((nx,ny),order='F')

    # Define some strings to help us format the output javascript-friendly way
    var_name = "eta_%s" %(event)
    header   = "var %s = {\"width\":%d,\"height\":%d,\"values\":[" %(var_name, nx, ny)
    footer   = "]};\n"
    
    # Write the text to a file
    # We want the first event to overwrite the previous file
    if first_event:
        with open(out_file,'w') as f:
            f.write(header)
            first_flag = True
            for entry in Hmax.ravel():
                if first_flag:
                    f.write(str(entry))
                    first_flag = False
                else:
                    f.write(',' + str(entry))
            f.write(footer)
        first_event = False
    # Otherwise we just append incoming data as new rows in the same file
    else:
        with open(out_file,'a') as f:
            f.write(header)
            first_flag = True
            for entry in Hmax.ravel():
                if first_flag:
                    f.write(str(entry))
                    first_flag = False
                else:
                    f.write(',' + str(entry))
            f.write(footer)
        